In [1]:
import pandas as pd
from transformers import AutoTokenizer
import gdown
import pandas as pd
import os
import re

In [2]:
!pip install trl accelerate -i https://pypi.org/simple/ bitsandbytes peft datasets # youll need these loaded up later on


Looking in indexes: https://pypi.org/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 36.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.4/102.4 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 21.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py

In [2]:
import gspread
from google.colab import auth
import pandas as pd
from google.auth import default

# Step 1: Authenticate
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# Step 2: Open the Google Sheets document
spreadsheet_url = 'https://docs.google.com/spreadsheets/d/1cmcfmaSxqA8Y4JkIuYZ0PsBxBT-Q_8XVNlZFsaFHfTI/edit?usp=sharing'

# Extract the spreadsheet ID from the URL
spreadsheet_id = spreadsheet_url.split('/')[5]

# Open the spreadsheet by its ID
spreadsheet = gc.open_by_key(spreadsheet_id)

# Step 3: Select the first sheet
worksheet = spreadsheet.get_worksheet(0)

# Step 4: Get all values in the sheet as a list of lists
data = worksheet.get_all_values()

# Step 5: Convert to a pandas DataFrame
df = pd.DataFrame(data)

# Optional: Set the first row as the header
df.columns = df.iloc[0]
df = df[1:]

In [3]:
df['Body'] = df['Body'] + ',\nIan'

In [4]:
df = df.reset_index(drop = True)

In [5]:
len(df)

177

In [7]:
print(df['Body'][0])

Good afternoon,

I hope this email finds you well :) My housemate and I are current students
at the University of San Francisco, and we are very interested in Judith
Butler's upcoming 06-13 lecture. Would it be possible to request two
student tickets for the event?

Attached, please find screenshots of both of our student IDs. Thank you so
much in advance!

With care,
Ian


# Load + configure Llama3 (8B params) and tokenize email body

In [ ]:
token = # add token here

In [ ]:
from huggingface_hub import login
login(token = token, add_to_git_credential=True)

Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
os.environ["HF_TOKEN"] = token

In [ ]:
# Select the model
model_name = "meta-llama/Meta-Llama-3-8B"

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=os.environ["HF_TOKEN"])

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:769: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
df['Body'] = df['Body'].astype(str)

# Define the tokenize function
def tokenize_function(text):
    return tokenizer(text, padding="max_length", truncation=True, return_tensors="pt")

# Tokenize the 'Body' column
df['tokenized_Body'] = df['Body'].apply(lambda x: tokenize_function(x))

# Display the tokenized data
print(df[['Date', 'tokenized_Body']].head())

Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


0                       Date               tokenized_Body
0  2024-06-06 15:34:13-07:00  [input_ids, attention_mask]
1  2024-06-05 10:59:41-07:00  [input_ids, attention_mask]
2  2024-06-04 12:54:23-07:00  [input_ids, attention_mask]
3  2024-06-03 21:16:05-07:00  [input_ids, attention_mask]
4  2024-05-29 09:51:46-07:00  [input_ids, attention_mask]


In [ ]:
# from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from trl import SFTTrainer
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
target_modules = ["q_proj", "v_proj"]

In [ ]:
from transformers import BitsAndBytesConfig
from transformers import AutoModelForCausalLM, AutoTokenizer


quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type='nf4'
)


model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map='auto',
    # torch_dtype=torch.bfloat16,
    quantization_config=quantization_config,
    use_auth_token=os.environ["HF_TOKEN"]
)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:468: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/177 [00:00<?, ?B/s]

# Fine tune using Ian's emails

In [ ]:
from sklearn.model_selection import train_test_split

# Lets create a simple 80 20 train-test split
train_df, eval_df = train_test_split(df, test_size=0.2, random_state=42)

eval_df = eval_df.reset_index(drop = True)
train_df = train_df.reset_index(drop = True)

In [ ]:
# Create a HuggingFace friendly dataframe
df_hug = pd.DataFrame({
    "Body": df['Body'],
    "tokenized_Body": df['tokenized_Body']
})

train_df = pd.DataFrame({
    "Body": train_df['Body'],
    "tokenized_Body": train_df['tokenized_Body']
})

eval_df = pd.DataFrame({
    "Body": eval_df['Body'],
    "tokenized_Body": eval_df['tokenized_Body']
})

In [ ]:
eval_df[:2]

,Body,tokenized_Body
0,"Hello,\n\nI hope this email finds you well. \n...","[input_ids, attention_mask]"
1,"Hey Dylan,\n\nThanks for passing these along. ...","[input_ids, attention_mask]"


In [ ]:
email_bodies = df['Body'].tolist()
email_bodies_train = train_df['Body'].tolist()
email_bodies_eval = eval_df['Body'].tolist()

# Tokenizing the email bodies
tokenized_bodies = [tokenizer.encode(body) for body in email_bodies]
tokenized_bodies_train = [tokenizer.encode(body) for body in email_bodies_train]
tokenized_bodies_eval = [tokenizer.encode(body) for body in email_bodies_eval]


# Create a DataFrame
df = pd.DataFrame({
    "Body": email_bodies,
    "tokenized_Body": tokenized_bodies
})

df_train = pd.DataFrame({
    "Body": email_bodies_train,
    "tokenized_Body": tokenized_bodies_train
})

df_eval = pd.DataFrame({
    "Body": email_bodies_eval,
    "tokenized_Body": tokenized_bodies_eval
})

In [ ]:
df_eval[:2]

,Body,tokenized_Body
0,"Hello,\n\nI hope this email finds you well. \n...","[128000, 9906, 3638, 40, 3987, 420, 2613, 1403..."
1,"Hey Dylan,\n\nThanks for passing these along. ...","[128000, 19182, 44458, 3638, 12947, 369, 12579..."


Yay! Looks great

In [ ]:
from datasets import Dataset

# Convert the DataFrame to a Hugging Face Dataset
dataset = Dataset.from_pandas(df)
dataset_train = Dataset.from_pandas(df_train)
dataset_eval = Dataset.from_pandas(df_eval)


# Convert tokenized bodies to the required format
def convert_to_features(example):
    return {
        "input_ids": example["tokenized_Body"],
        "attention_mask": [1] * len(example["tokenized_Body"])  # Assuming all tokens are valid
    }

# Apply the conversion to the dataset
tokenized_dataset = dataset.map(convert_to_features, remove_columns=["Body"])
tokenized_dataset_train = dataset_train.map(convert_to_features, remove_columns=["Body"])
tokenized_dataset_eval = dataset_eval.map(convert_to_features, remove_columns=["Body"])


Map:   0%|          | 0/177 [00:00<?, ? examples/s]

Map:   0%|          | 0/141 [00:00<?, ? examples/s]

Map:   0%|          | 0/36 [00:00<?, ? examples/s]

In [ ]:
tokenized_dataset_train

Dataset({
    features: ['tokenized_Body', 'input_ids', 'attention_mask'],
    num_rows: 141
})

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments
from datasets import Dataset
import torch
from peft import LoraConfig, get_peft_model, TaskType

# Define LoRA configuration for PEFT
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=8,  # Rank of the low-rank adaptation matrix
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none"
)

# model.add_adapter(lora_config)

# Attach LoRA adapters to the model
peft_model = get_peft_model(model, lora_config)
peft_model.to("cuda" if torch.cuda.is_available() else "cpu")

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=1,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',  # Specify the directory for logging
    logging_steps=10,  # Log every 10 steps
)

# Create Trainer instance
trainer = SFTTrainer(
    model=peft_model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_dataset_eval
)

# Fine-tune the model
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1965: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_token` instead.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:278: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,3.020700,2.783458
2,2.748700,2.641151
3,2.602300,2.613336


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


TrainOutput(global_step=531, training_loss=2.934934587352918, metrics={'train_runtime': 1422.0126, 'train_samples_per_second': 0.373, 'train_steps_per_second': 0.373, 'total_flos': 2471217490206720.0, 'train_loss': 2.934934587352918, 'epoch': 3.0})

In [ ]:
question = "Please write a single email to my professor, Nico, requesting to miss lecture."
print(f"Question: {question}")

def formatting_prompts_func(example):
    output_texts = []
    for i in range(len(example['email request'])):
        text = f"Respond to this request:\n\n{example['email request'][i]}.\n\nResponse:"
        output_texts.append(text)
    return output_texts


def get_outputs(model, inputs, max_new_tokens=100):
    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_new_tokens=max_new_tokens,
        repetition_penalty=1.5,
        early_stopping=True,
        eos_token_id=tokenizer.eos_token_id,
    )
    return outputs

# Formatting the prompt
formatted_prompt = formatting_prompts_func({
    "email request": [question],
    "answer": [""],
})

# Check the output format of the formatting_prompts_func
print(f"Formatted Prompt: {formatted_prompt}")

# Ensure the formatted prompt is a string
if isinstance(formatted_prompt, dict) and "email request" in formatted_prompt:
    formatted_prompt = formatted_prompt["email request"][0]

input_sentences = tokenizer(formatted_prompt, return_tensors="pt").to(device)

# Define get_outputs function if not already defined
def get_outputs(model, inputs, max_new_tokens):
    outputs = model.generate(**inputs, max_new_tokens=max_new_tokens)
    return outputs

foundational_outputs_sentence = get_outputs(model, input_sentences, max_new_tokens=100)

print(tokenizer.batch_decode(foundational_outputs_sentence, skip_special_tokens=True)[0])


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Question: Please write a single email to my professor, Nico, requesting to miss lecture.
Formatted Prompt: ['Respond to this request:\n\nPlease write a single email to my professor, Nico, requesting to miss lecture..\n\nResponse:']
Respond to this request:

Please write a single email to my professor, Nico, requesting to miss lecture..

Response: 

Hey Nico,

I hope you're doing well! I just wanted to let you know that I will not be able to attend lecture on Tuesday, May 10th at 12:30pm. I have a doctor's appointment that day and will be unable to make it to class. I will be back in time for the lab that afternoon, but will miss the lecture. Thanks so much!

,
Ian



In [ ]:
1